# Saving Algerian Constitution 2020 into SurrealDB

## 1. Importing libraries

In [2]:
%pip install surrealist einops openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datetime import datetime
from surrealist import Surreal
from openai import OpenAI
import json
import os

%reload_ext dotenv
%dotenv .env

## 2. Load the data

In [4]:
json_file_path = os.path.join("json", "dz_consti_2020.json")
with open(json_file_path, "r", encoding="utf-8") as f:
    constitution_data = json.load(f)

## 3. Connect to the database

In [5]:
surreal = Surreal("http://127.0.0.1:28900", credentials=(os.getenv("SURREAL_USER"), os.getenv("SURREAL_PASS"))) 
print(surreal.is_ready())
print(surreal.version())

True
surrealdb-2.1.0


## 4. Embed articles and save to database

In [9]:
model_name = "text-embedding-3-small"
client = OpenAI() # api key loaded from .env

def save_constitution():
    with surreal.connect() as db:
        db.use('ites', "test")
        db.query("DEFINE TABLE articles")
        titles = constitution_data['titles'][1:]
        for title in titles:
            chapters = title['chapters']
            for chapter in chapters:
                articles = chapter['articles']
                for article in articles:
                    # more about this on https://platform.openai.com/docs/guides/embeddings#how-to-get-embeddings
                    response = client.embeddings.create(
                        input=f"{title['header']}: {chapter['header']}: {article}",
                        model=model_name,
                    )
                    vector = response.data[0].embedding

                    db.insert(
                        "articles",
                        [{
                            "text": article,
                            "embed_vector": vector,
                            "chapter": chapter['header'],
                            "title": title['header'],
                            "edition": datetime(2020, 1, 1).strftime('%Y-%m-%dT%H:%M:%SZ'),
                        }]
                    )


save_constitution()

In [ ]:
user_question = "worshiping in Algeria"
# user_question = "Qu'est-ce que je peux faire dans la guerre?"

response = client.embeddings.create(
    input=user_question,
    model=model_name,
)
q_vector = response.data[0].embedding

with surreal.connect() as db:
    db.use('ites', "test")
    res = db.query("SELECT text, vector::similarity::cosine(embed_vector, $q_vector) AS sim FROM articles ORDER BY sim DESC LIMIT 10", {"q_vector": q_vector})

print(json.dumps(res.result, indent=2, ensure_ascii=False))

[
  {
    "sim": 0.21771088476828357,
    "text": "المادة 29 : تمتنع الجزائر عن اللّجوء إلى الحرب من أجل المساس بالسّيادة المشروعة للشّعوب الأخرى وحرّيّتها. وتبذل جهدها لتسوية الخلافات الدّوليّة بالوسائل السّلميّة."
  },
  {
    "sim": 0.19280549397366195,
    "text": "المادة 169 : الحقّ في الدّفاع معترف به. الحقّ في الدّفاع مضمون في القضايا الجزائيّة."
  },
  {
    "sim": 0.18122965020704287,
    "text": "المادة 28 : تنتظم الطّاقة الدّفاعيّة للأمّة، ودعمها، وتطويرها، حول الجيش الوطنيّ الشّعبيّ. تتمثّل المهمّة الدّائمة للجيش الوطنيّ الشّعبيّ في المحافظة على الاستقلال الوطنيّ، والدّفاع عن السّيادة الوطنيّة. كما يضطلع بالدّفاع عن وحدة البلاد، وسلامتها التّرابيّة، وحماية مجالها البرّيّ والجوّيّ، ومختلف مناطق أملاكها البحريّة."
  },
  {
    "sim": 0.17849468826703682,
    "text": "المادة 71  : الحقّ في الإضراب مُعترَف به، ويُمارَس في إطار القانون. يمكن أن يمنع القانون ممارسة هذا الحقّ، أو يجعل حدودا لممارسته في ميادين الدّفاع الوطنيّ والأمن، أو في جميع الخدمات أو الأعمال العموميّة ذات المن